### Categorize kanji

In [5]:
# imports
import pandas

from dataclasses import dataclass

# data classes
@dataclass        
class Kanji:
    rank: int
    name: str

In [6]:
# read excel file (with mapping)
df_kanji = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="MAIN")
df_kanji.columns = ["CHAR", "COMPONENTS1", "COMPONENTS2", "COMPONENTS3", "COMPONENTS4", "COMPONENTS5", "ON READING", "KUN READING", "KEYWORD", "SRL", "TYPE", "FREQ", "TAGS"]
df_kanji

,CHAR,COMPONENTS1,COMPONENTS2,COMPONENTS3,COMPONENTS4,COMPONENTS5,ON READING,KUN READING,KEYWORD,SRL,TYPE,FREQ,TAGS
0,一,NaN,NaN,NaN,NaN,NaN,イチ、イツ,ひと・つ,one,5,STEM,10.800000,NaN
1,二,NaN,NaN,NaN,NaN,NaN,ニ,ふた・つ,two,5,MEAN,128.300000,NaN
2,三,NaN,NaN,NaN,NaN,NaN,サン,みっ・つ,three,5,MEAN,120.700000,NaN
3,四,NaN,NaN,NaN,NaN,NaN,シ,よっ・つ、よん、よ,four,5,MEAN,312.073333,NaN
4,五,NaN,NaN,NaN,NaN,NaN,ゴ,いつ・つ,five,5,MEAN,315.626667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,気,气,㐅,NaN,NaN,NaN,キ、ケ,NaN,atmosphere,1,OTHER,64.000000,NaN
1496,風,𠘨,䖝,NaN,NaN,NaN,フウ、フ,かぜ,1 wind 2 style,1,OTHER,289.700000,NaN
1497,乳,⺤,子,乚,NaN,NaN,ニュウ,ち、ちち,milk,1,OTHER,1067.000000,NaN
1498,興,臼,同,ハ,NaN,NaN,コウ、キョウ,NaN,interest,1,OTHER,649.600000,NaN


In [7]:
df_keyword = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="keyword.list")
# df_keyword

In [8]:
df_stem = pandas.read_excel("1500 KANJI COMPONENTS - ver. 1.2.xlsx", sheet_name="stem.list")
# df_stem

### Algorithm

In [13]:
random_row = df_kanji[df_kanji["CHAR"] == '五'].iloc[0]
random_row

CHAR                    五
COMPONENTS1           NaN
COMPONENTS2           NaN
COMPONENTS3           NaN
COMPONENTS4           NaN
COMPONENTS5           NaN
ON READING              ゴ
KUN READING          いつ・つ
KEYWORD              five
SRL                     5
TYPE                 MEAN
FREQ           315.626667
TAGS                  NaN
Name: 4, dtype: object

In [9]:
categorization = {}
special_grp = '78 special'
other_grp = '77 other'

for grp in df_keyword["GROUP"].unique():
    categorization[grp] = []

for grp in df_stem["GROUP"].unique():
    categorization[grp] = []

categorization[other_grp] = []
categorization[special_grp] = []

# categorization["1 numbers"] = []
# categorization["1 numbers"].append(Test("aa", "bb"))
# categorization["1 numbers"].append(Test("ca", "ab"))
# categorization["1 numbers"].insert(0, Test("xxx", "ab"))

categorization

{'1 numbers': [],
 '2 family': [],
 '3 colors': [],
 '31 cardinals': [],
 '32 season of the year': [],
 '33 parts of the day': [],
 '77 other': [],
 '4 Sunday': [],
 '5 Monday': [],
 '6 Tuesday': [],
 '7 flame': [],
 '8 Wednesday': [],
 '9 alcohol': [],
 '10 Thursday': [],
 '11 Friday': [],
 '12 Saturday': [],
 '13 human': [],
 '14 woman': [],
 '15 child': [],
 '16 ear': [],
 '17 eye': [],
 '18 to see': [],
 '19 mouth': [],
 '20 tongue': [],
 '21 talk': [],
 '22 beard': [],
 '23 heart': [],
 '24 arm': [],
 '25 leg': [],
 '26 kneel': [],
 '27 stand': [],
 '28 run': [],
 '29 route': [],
 '30 direction': [],
 '32 seasons of the year': [],
 '34 car': [],
 '35 ship': [],
 '36 rain': [],
 '37 gate': [],
 '38 roof A': [],
 '39 roof B': [],
 '40 roof C': [],
 '41 roof D': [],
 '42 temple': [],
 '43 insect': [],
 '44 dog': [],
 '45 hound': [],
 '46 sheep': [],
 '47 cow': [],
 '48 horse': [],
 '49 bird': [],
 '50 feather': [],
 '51 field': [],
 '52 mound': [],
 '53 grass': [],
 '54 thread': [],


In [14]:
# first rule (check keyword)
def find_keyword(row):
    group = df_keyword[df_keyword["KEYWORD"] == row["KEYWORD"]]["GROUP"]
    if group.empty:
        return "none"
    else:
        return group.iloc[0]

def find_stem(row):
    group = df_stem[df_stem["STEM KANJI"] == row["CHAR"]]["GROUP"]
    if group.empty:
        return "none"
    else:
        return group.iloc[0]

first_rule = find_keyword(random_row)
second_rule = find_stem(random_row)
if (first_rule != "none"):
    print("1. rule")
    if (random_row["TYPE"] == "MEAN"):
        categorization[first_rule].append(random_row)
    else:
        if (random_row["TYPE"] == "SPECIAL"):
            categorization[special_grp].append(random_row)
        else:
            if (random_row["TYPE"] == "OTHER"):
                categorization[other_grp].append(random_row)
            else:
                print("ERROR: missing grp")
else:
    if (second_rule != "none"):
        print("2. rule")
        if (random_row["TYPE"] == "STEM"):
            categorization[second_rule].insert(0, random_row)
        else:
            print("ERROR: missing rule")
    else:
        components = df_kanji[(df_kanji["COMPONENTS1"] == random_row["CHAR"]) | (df_kanji["COMPONENTS2"] == random_row["CHAR"])]
        if components.empty:
            print("nope")
        else:
            print("3.rule")
            onyomi = components[components["ON READING"].str.ns(randow_row["ON READING"])]
            if onyomi.empty:
                print("4.rule")
            else:
                if onyomi.size > 1:
                    
    
categorization
# find_keyword(random_row)
# find_stem(random_row)

1. rule


{'1 numbers': [CHAR                    五
  COMPONENTS1           NaN
  COMPONENTS2           NaN
  COMPONENTS3           NaN
  COMPONENTS4           NaN
  COMPONENTS5           NaN
  ON READING              ゴ
  KUN READING          いつ・つ
  KEYWORD              five
  SRL                     5
  TYPE                 MEAN
  FREQ           315.626667
  TAGS                  NaN
  Name: 4, dtype: object],
 '2 family': [],
 '3 colors': [],
 '31 cardinals': [],
 '32 season of the year': [],
 '33 parts of the day': [],
 '77 other': [],
 '4 Sunday': [],
 '5 Monday': [],
 '6 Tuesday': [],
 '7 flame': [],
 '8 Wednesday': [],
 '9 alcohol': [],
 '10 Thursday': [],
 '11 Friday': [],
 '12 Saturday': [],
 '13 human': [],
 '14 woman': [],
 '15 child': [],
 '16 ear': [],
 '17 eye': [],
 '18 to see': [],
 '19 mouth': [],
 '20 tongue': [],
 '21 talk': [],
 '22 beard': [],
 '23 heart': [],
 '24 arm': [],
 '25 leg': [],
 '26 kneel': [],
 '27 stand': [],
 '28 run': [],
 '29 route': [],
 '30 direction': [],
 

In [53]:
# df[df["COMPONENTS1"] == "c"]
# mb = mb.dropna(subset=['ON READING'])
# mb[mb["ON READING"].str.ns("フク")]

,CHAR,COMPONENTS1,COMPONENTS2,COMPONENTS3,COMPONENTS4,COMPONENTS5,COMPONENTS6,ON READING,KUN READING,KEYWORD,SRL,TYPE,FREQ
80,郷,幺,艮,阝,NaN,NaN,NaN,キョウ,NaN,hometown,1,VR,1029.706667
